In [2]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82

In [3]:
import nltk
import evaluate
import numpy as np
import torch

from datasets import load_dataset, DatasetDict
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

from sklearn.model_selection import train_test_split


In [4]:

# Load the tokenizer and model
MODEL_NAME = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Load the Movies dataset
dataset = load_dataset("wykonos/movies")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

movies_dataset.csv:   0%|          | 0.00/350M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/722796 [00:00<?, ? examples/s]

In [6]:

# Function to format data into text-to-text format
def format_example(example):
    input_text = f"Generate a movie: Genre: {example['genres']}, Production Company: {example['production_companies']}, Year: {example['release_date']}"
    output_text = f"Title: {example['title']}, Genre: {example['genres']}, Production Company: {example['production_companies']}, Year: {example['release_date']}, Key words: {example['keywords']}"
    return {"input_text": input_text, "output_text": output_text}


In [7]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples["output_text"], max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [8]:
# Convert dataset to pandas for easier splitting
df = dataset["train"].to_pandas()

# Perform train-test split (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert back to Hugging Face datasets
train_dataset = DatasetDict({"train": dataset["train"].from_pandas(train_df)})
test_dataset = DatasetDict({"test": dataset["train"].from_pandas(test_df)})

In [9]:
## applying formatting
train_dataset = train_dataset.map(format_example)
test_dataset = test_dataset.map(format_example)

Map:   0%|          | 0/578236 [00:00<?, ? examples/s]

Map:   0%|          | 0/144560 [00:00<?, ? examples/s]

In [10]:
# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset["train"].column_names)
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset["test"].column_names)


Map:   0%|          | 0/578236 [00:00<?, ? examples/s]

Map:   0%|          | 0/144560 [00:00<?, ? examples/s]

In [17]:
## let's disable weights and biases (wandb) library, it requires API

import os
os.environ["WANDB_DISABLED"] = "true"
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-movies",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=500,
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset["train"],
    eval_dataset=test_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-17-f0b30131d9d5>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [13]:

## WANDB was disabled above Trainer

In [18]:

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [20]:

# Save model and tokenizer
model.save_pretrained("./t5-movies")
tokenizer.save_pretrained("./t5-movies")

('./t5-movies/tokenizer_config.json',
 './t5-movies/special_tokens_map.json',
 './t5-movies/spiece.model',
 './t5-movies/added_tokens.json')

In [25]:
def generate_movie(genre, prod_company, year):
    input_text = f"Generate a movie: Genre: {genre}, Production Company: {prod_company}, Year: {year}"

    # Move tokenizer inputs to the same device as the model (CUDA)
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Ensure the model is on CUDA
    model.to("cuda")
      # Generate output
    with torch.no_grad():
        output = model.generate(**inputs, max_length=256)

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [27]:
print(generate_movie("Animation-Family", "Universal Pictures", "2023"))

Title: The Big Picture, Genre: Animation-Family, Production Company: Universal Pictures, Year: 2023, Key words: None


In [30]:
print(generate_movie("Drama", "Nordisk Film Norway", "2022"))

Title: The Last Sword, Genre: Drama, Production Company: Nordisk Films Norway, Year: 2022, Key words: None
